In [38]:
import pandas as pd
from pathlib import Path
import os
import html

Data processing

In [39]:
PO_GOO_DIR = Path('po-en-goo')
PO_DL_DIR = Path('po-en-dl')
FR_GOO_DIR = Path('fr-en-goo')
FR_DL_DIR = Path('fr-en-dl')
EN1_DIR = Path('en1')
EN2_DIR = Path('en2')

DIRS = [PO_GOO_DIR, PO_DL_DIR, FR_GOO_DIR, FR_DL_DIR, EN1_DIR, EN2_DIR]

In [40]:
# Find out the replacement by checking the vocabularies in parellel chapters for confirmation
TO_REPLACE = {
    'ö': 'o',
    'é': 'e',
    'ë': 'e',
    'ï': 'i',
    'ą': 'a',
    'í': 'i',
    'æ': 'ae',
    'ð': 'ny', # Original - fr-dl[8] 'caðon', changed to based on po-en-goo 'Canyon'
    'à': 'a',
    'å': 'a',
    'â': 'a',
    
}

In [41]:

allowed="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 \n"
def get_not_allowed(txt):
    others = set()
    for i in txt:
        if i not in allowed:
            others.add(i)
    return others

In [42]:
# Check special characters by checking each translated texts.
others = set()
for i in EN2_DIR.iterdir():
    with open(i) as f:
        txt = f.read()
        # Change the html punctuation format to the normal one
        txt = html.unescape(txt)
    others = others.union(get_not_allowed(txt))
print(others)

{"'", '.', '”', '!', 'å', '“', ':', 'â', ')', ',', '?', 'ö', '(', '’', 'é', '–', ';', '—', '-', '‘', '\x0c'}
